In [10]:
from dotenv import load_dotenv
load_dotenv('azure.env', override=True)

True

In [11]:
os.environ["PROJECT_CONNECTION_STRING"]

'swedencentral.api.azureml.ms;f17b9f1d-4088-479a-acb6-6dbf870ca4dc;openai-sweden;mrajguru-4522'

In [12]:
os.environ["MODEL_DEPLOYMENT_NAME"]

'gpt-4o-sweden'

In [13]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import (
    AgentEventHandler,
    MessageDeltaChunk,
    ThreadMessage,
    ThreadRun,
    RunStep,
)
from typing import Any
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [14]:



class MyEventHandler(AgentEventHandler):
    def on_message_delta(self, delta: "MessageDeltaChunk") -> None:
        print(f"Text delta received: {delta.text}")

    def on_thread_message(self, message: "ThreadMessage") -> None:
        if len(message.content):
            print(
                f"ThreadMessage created. ID: {message.id}, "
                f"Status: {message.status}, Content: {message.content[0].as_dict()}"
            )
        else:
            print(f"ThreadMessage created. ID: {message.id}, " f"Status: {message.status}")

    def on_thread_run(self, run: "ThreadRun") -> None:
        print(f"ThreadRun status: {run.status}")

    def on_run_step(self, step: "RunStep") -> None:
        print(f"RunStep type: {step.type}, Status: {step.status}")

    def on_error(self, data: str) -> None:
        print(f"An error occurred. Data: {data}")

    def on_done(self) -> None:
        print("Stream completed.")

    def on_unhandled_event(self, event_type: str, event_data: Any) -> None:
        print(f"Unhandled Event Type: {event_type}, Data: {event_data}")


# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)

scenario = "tracing"
tracer = trace.get_tracer(__name__)

# enable additional instrumentations
project_client.telemetry.enable()

with tracer.start_as_current_span(scenario):
    with project_client:
        # Create an agent and run stream with event handler
        agent = project_client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"], name="my-assistant", instructions="You are a helpful assistant"
        )
        print(f"Created agent, agent ID {agent.id}")

        thread = project_client.agents.create_thread()
        print(f"Created thread, thread ID {thread.id}")

        message = project_client.agents.create_message(
            thread_id=thread.id, role="user", content="Hello, tell me a joke"
        )
        print(f"Created message, message ID {message.id}")

        with project_client.agents.create_stream(
            thread_id=thread.id, agent_id=agent.id, event_handler=MyEventHandler()
        ) as stream:
            stream.until_done()

        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        messages = project_client.agents.list_messages(thread_id=thread.id)
        print(f"Messages: {messages}")

Created agent, agent ID asst_4ARwRR5SDmeLL6pwPoGHAUyz
Created thread, thread ID thread_5KHMsVZxk72kGwd9kYWTRllZ
Created message, message ID msg_hBswuzYHwevZq2QmmIIHTiOI
ThreadRun status: RunStatus.QUEUED
ThreadRun status: RunStatus.QUEUED
ThreadRun status: RunStatus.IN_PROGRESS
RunStep type: RunStepType.MESSAGE_CREATION, Status: RunStepStatus.IN_PROGRESS
RunStep type: RunStepType.MESSAGE_CREATION, Status: RunStepStatus.IN_PROGRESS
ThreadMessage created. ID: msg_1xB5IR4eUUEUkc23oOLOEESB, Status: MessageStatus.IN_PROGRESS
ThreadMessage created. ID: msg_1xB5IR4eUUEUkc23oOLOEESB, Status: MessageStatus.IN_PROGRESS
Text delta received: Sure
Text delta received: !
Text delta received:  Here's
Text delta received:  a
Text delta received:  joke
Text delta received:  for
Text delta received:  you
Text delta received: :


Text delta received: Why
Text delta received:  can't
Text delta received:  you
Text delta received:  give
Text delta received:  Elsa
Text delta received:  a
Text delta received: